
# Taller de Visualización y Análisis Exploratorio — Seguros (VSC + Jupyter)

**Dataset:** `seguros.csv` (columnas: `age`, `sex`, `bmi`, `children`, `smoker`, `region`, `charges`)

Este cuaderno está diseñado para ejecutarse en **Visual Studio Code** con la extensión de **Jupyter**.
Incluye pasos guiados, buenas prácticas, múltiples gráficos y **retos** para practicar.

> Sugerencia: Coloca este `.ipynb` en la misma carpeta donde está `seguros.csv`.


## 1) Preparación del entorno

In [ ]:

# Si necesitas instalar paquetes (ejecuta una sola vez desde una celda):
# !pip install pandas matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Opcional: estilo por defecto (seaborn)
sns.set()
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


## 2) Carga del dataset

In [ ]:

# Ruta al archivo
PATH = "seguros.csv"  # Ajusta si tu archivo está en otra ruta

# Cargar
df = pd.read_csv(PATH)

# Vistazo inicial
print(df.shape)
df.head()


## 3) Exploración inicial: tipos, nulos, duplicados, estadísticos

In [ ]:

df.info()


In [ ]:

# Nulos y duplicados
print("Nulos por columna:")
print(df.isna().sum())

print("\nDuplicados:", df.duplicated().sum())


In [ ]:

# Estadísticos descriptivos básicos
df.describe(include="all")



## 4) Limpieza mínima y variables derivadas

- Normalizamos categorías (opcional).
- Creamos categorías de IMC (OMS) para segmentar gráficos.


In [ ]:

# Asegurar minúsculas en categorías
df["sex"] = df["sex"].str.lower()
df["smoker"] = df["smoker"].str.lower()
df["region"] = df["region"].str.lower()

# Buckets de BMI según OMS aproximado
def bmi_bucket(bmi):
    if bmi < 18.5: return "Bajo peso"
    if bmi < 25:   return "Normal"
    if bmi < 30:   return "Sobrepeso"
    return "Obesidad"

df["bmi_cat"] = df["bmi"].apply(bmi_bucket)
df.head()


## 5) Visualizaciones univariadas

In [ ]:

# Histograma de 'charges'
plt.figure()
plt.hist(df["charges"], bins=30)
plt.title("Distribución de 'charges'")
plt.xlabel("charges"); plt.ylabel("frecuencia")
plt.show()


In [ ]:

# Histograma de 'bmi'
plt.figure()
plt.hist(df["bmi"], bins=30)
plt.title("Distribución de 'bmi'")
plt.xlabel("bmi"); plt.ylabel("frecuencia")
plt.show()


In [ ]:

# Barras de conteo para categóricas
plt.figure()
df["smoker"].value_counts().plot(kind="bar")
plt.title("Conteo de fumadores vs no fumadores")
plt.xlabel("smoker"); plt.ylabel("conteo")
plt.show()


## 6) Visualizaciones bivariadas

In [ ]:

# Dispersión: bmi vs charges
plt.figure()
plt.scatter(df["bmi"], df["charges"], alpha=0.6)
plt.title("bmi vs charges")
plt.xlabel("bmi"); plt.ylabel("charges")
plt.show()


In [ ]:

# Boxplot: charges por smoker
plt.figure()
df.boxplot(column="charges", by="smoker")
plt.title("Charges por 'smoker'"); plt.suptitle("")
plt.xlabel("smoker"); plt.ylabel("charges")
plt.show()


In [ ]:

# Boxplot: charges por region
plt.figure()
df.boxplot(column="charges", by="region", rot=0)
plt.title("Charges por 'region'"); plt.suptitle("")
plt.xlabel("region"); plt.ylabel("charges")
plt.show()


## 7) Visualizaciones multivariadas

In [ ]:

# Facetas: relación bmi-charges, separado por 'smoker'
g = sns.FacetGrid(df, col="smoker", height=4)
g.map(plt.scatter, "bmi", "charges", alpha=0.6)
g.add_legend()
plt.show()


In [ ]:

# Facetas 2: charges vs age por categoría de bmi
g = sns.FacetGrid(df, col="bmi_cat", col_wrap=2, height=3.5, sharey=False)
g.map(plt.scatter, "age", "charges", alpha=0.6)
g.add_legend()
plt.show()


In [ ]:

# Matriz de correlación (numéricas)
num_cols = ["age", "bmi", "children", "charges"]
corr = df[num_cols].corr()
plt.figure()
plt.imshow(corr, cmap="viridis")
plt.colorbar()
plt.xticks(range(len(num_cols)), num_cols, rotation=45)
plt.yticks(range(len(num_cols)), num_cols)
plt.title("Matriz de correlación (numéricas)")
for i in range(len(num_cols)):
    for j in range(len(num_cols)):
        plt.text(j, i, f"{corr.iloc[i,j]:.2f}", ha="center", va="center")
plt.tight_layout()
plt.show()


## 8) Filtros y segmentaciones (query / loc)

In [ ]:

# Filtro: fumadores con BMI > 30
f1 = df.query("smoker == 'yes' and bmi > 30")
f1.head()


In [ ]:

# Filtro múltiple: región 'southeast' y al menos 2 hijos
f2 = df[(df["region"] == "southeast") & (df["children"] >= 2)]
f2.head()


## 9) Agrupaciones (groupby), comparaciones y tablas pivote

In [ ]:

# Promedio de charges por smoker
grp1 = df.groupby("smoker")["charges"].agg(["count","mean","median","std"]).reset_index()
grp1


In [ ]:

# Promedio de charges por región y fumador
grp2 = df.groupby(["region","smoker"])["charges"].mean().unstack()
grp2


In [ ]:

# Tabla pivote: media de charges por (region x bmi_cat)
pv = pd.pivot_table(df, values="charges", index="region", columns="bmi_cat",
                    aggfunc="mean", fill_value=0)
pv


In [ ]:

# Gráfico de barras apiladas a partir de la pivote (media charges por region x bmi_cat)
plt.figure()
pv.plot(kind="bar", stacked=True)
plt.title("Media de charges por región y categoría de BMI")
plt.xlabel("region"); plt.ylabel("media de charges")
plt.tight_layout()
plt.show()


## 10) Combinaciones útiles: grid de gráficos

In [ ]:

# Grid 2x2 con diferentes vistas de 'charges'
fig, axes = plt.subplots(2,2, figsize=(10,8))

axes[0,0].hist(df["charges"], bins=30)
axes[0,0].set_title("Hist charges")

axes[0,1].scatter(df["bmi"], df["charges"], alpha=0.5)
axes[0,1].set_title("bmi vs charges")

df_seguros.boxplot(column="charges", by="smoker", ax=axes[1,0])
axes[1,0].set_title("charges por smoker")
axes[1,0].figure.suptitle("")

df_seguros.boxplot(column="charges", by="region", ax=axes[1,1])
axes[1,1].set_title("charges por region")
axes[1,1].figure.suptitle("")

plt.tight_layout()
plt.show()


## 11) Guardar figuras en disco (para informes)

In [ ]:

import os
os.makedirs("figs", exist_ok=True)

plt.figure()
plt.hist(df["charges"], bins=30)
plt.title("Distribución de charges")
plt.savefig("figs/hist_charges.png", dpi=150, bbox_inches="tight")
plt.close()

print("Figura guardada en figs/hist_charges.png")



## 12) Retos (para practicar)

1. **Comparar impacto del tabaquismo por región:**  
   Calcula `mean`, `median` y `std` de `charges` por `smoker` dentro de cada `region`.  
   Luego, grafica un **bar chart** de las medias para comparar.

2. **Efecto de la edad por categorías de IMC:**  
   Crea grupos de edad (por ejemplo: `<=30`, `31-45`, `>45`) y calcula la media de `charges` por (`bmi_cat`, `grupo_edad`).  
   Grafica con barras agrupadas.

3. **Top 10 clientes por charges** y su perfil:  
   Muestra un `df.nlargest(10, "charges")` con `age`, `bmi`, `smoker`, `region`.  
   Haz un gráfico de barras horizontal con sus `charges`.

4. **Detección simple de outliers en charges:**  
   Calcula Q1, Q3 e IQR; considera outlier si `charges < Q1 - 1.5*IQR` o `> Q3 + 1.5*IQR`.  
   ¿Qué porcentaje de registros resulta outlier?

5. **Mapa de calor simple de medias:**  
   Usa una tabla pivote de `charges` por `smoker` y `region` y plótala con `plt.imshow`.  
   Añade anotaciones con los valores.

> Tip: evita “hardcodear” categorías; usa `df["col"].unique()` para que tu código sea robusto.


## 13) Bonus: Funciones reutilizables / pipeline simple

In [ ]:

def resumen_num(df, cols):
    return df[cols].describe()

def graf_boxplot_por_cat(df, col_num, col_cat):
    plt.figure()
    df.boxplot(column=col_num, by=col_cat, rot=0)
    plt.title(f"{col_num} por {col_cat}"); plt.suptitle("")
    plt.show()

# Ejemplo de uso:
resumen_num(df, ["age","bmi","children","charges"])
# graf_boxplot_por_cat(df, "charges", "region")



---

### Siguientes pasos sugeridos
- Replicar este análisis con un subconjunto filtrado (por ejemplo, solo `smoker == "yes"`).
- Exportar métricas agregadas a CSV para un informe.
- Crear un **README** con capturas de los gráficos generados.

> ¿Listo para más? Prueba integrar este EDA con un **modelo simple** (regresión lineal) para explicar `charges` en función de `age`, `bmi`, `smoker`, etc.


In [1]:
# Filtrar solo fumadores
df_smokers = df_seguros[df_seguros["smoker"] == "yes"]

print(f"Total registros: {len(df_seguros)}")
print(f"Total fumadores: {len(df_smokers)}")

# Ejemplo: histograma de charges solo para fumadores
plt.figure(figsize=(7,4))
sns.histplot(df_smokers["charges"], kde=True, bins=20)
plt.title("Distribución de Charges - Fumadores")
plt.xlabel("Charges")
plt.ylabel("Frecuencia")
plt.show()

# Ejemplo: boxplot por región solo para fumadores
plt.figure(figsize=(7,4))
sns.boxplot(x="region", y="charges", data=df_smokers)
plt.title("Charges por región - Fumadores")
plt.show()

NameError: name 'df_seguros' is not defined